https://www.kaggle.com/competitions/vehicle/overview

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%mkdir /content/train

mkdir: cannot create directory ‘/content/train’: File exists


In [6]:
!unzip "/content/drive/MyDrive/BicycleImages.zip" -d "/content/train"
!unzip "/content/drive/MyDrive/MotorcycleImages.zip" -d "/content/train"

Archive:  /content/drive/MyDrive/BicycleImages.zip
replace /content/train/Bicycle/000655_02.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/drive/MyDrive/MotorcycleImages.zip
replace /content/train/Motorcycle/002593_00.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#!pip install kaggle --upgrade

In [ ]:
#from google.colab import files
#files.upload()
# 파일 선택 버튼 클릭 후 kaggle.json 파일 업로드

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"seongjuny","key":"e478989b7306d17378979f4497b5ac9f"}'}

In [ ]:
#!mkdir -p ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#!ls -1ha kaggle.json

kaggle.json


In [ ]:
#!kaggle competitions download -c vehicle

100% 6.88G/6.88G [00:54<00:00, 169MB/s]
100% 6.88G/6.88G [00:54<00:00, 135MB/s]


In [ ]:
# 확인
#!ls
# 압축 풀기
#!unzip vehicle.zip

In [ ]:
# import json

# # Load JSON data
# with open('/content/drive/MyDrive/ai_project/train/train.json', 'r') as file:
#     data = json.load(file)


In [4]:
import os
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray, rgb2hsv
from skimage.transform import resize
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler


In [5]:
# Define parameters for LBP
radius = 3
n_points = 8 * radius

def extract_lbp_features(image):
    lbp = local_binary_pattern(image, n_points, radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)  # Normalize the histogram
    return hist

def extract_color_histogram(image):
    hsv_image = rgb2hsv(image)
    hist, _ = np.histogram(hsv_image[:,:,0], bins=8, range=(0, 1))  # Histogram for hue channel
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)  # Normalize the histogram
    return hist


In [6]:
import os
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray  # Import rgb2gray
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

# Directory paths
bicycle_dir = '/content/train/Bicycle'
motorcycle_dir = '/content/train/Motorcycle'

# Image processing parameters
img_size = (480, 480)  # Example: Resize images to 128x128 pixels

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = imread(os.path.join(folder, filename))
        if img is not None:
            img_resized = resize(img, img_size, anti_aliasing=True)
            gray_img = rgb2gray(img_resized)
            gray_features = gray_img.flatten()
            lbp_features = extract_lbp_features(gray_img)
            color_hist_features = extract_color_histogram(img_resized)
            combined_features = np.hstack((gray_features, lbp_features, color_hist_features))  # Combine features
            images.append(combined_features)
            labels.append(label)
    return images, labels


In [7]:
# Load and label images
bicycle_images, bicycle_labels = load_images_from_folder(bicycle_dir, 0)
motorcycle_images, motorcycle_labels = load_images_from_folder(motorcycle_dir, 1)

# Combine data and labels
X = np.array(bicycle_images + motorcycle_images)
y = np.array(bicycle_labels + motorcycle_labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train SVM
svm = SVC(kernel='rbf', C=1000)
svm.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.72      0.57      0.64       355
           1       0.76      0.86      0.81       566

    accuracy                           0.75       921
   macro avg       0.74      0.72      0.72       921
weighted avg       0.75      0.75      0.74       921

